In [116]:
import json

In [117]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span

In [118]:
import json
from collections import defaultdict
from spacy.matcher import PhraseMatcher

In [119]:
import os
import glob

In [120]:
from collections import defaultdict

In [121]:
nlp = spacy.load('en_core_web_lg')

In [122]:
os.getcwd()

'/home/ec2-user/spacy-projects/tutorials/rel_component/scripts'

In [123]:
file_list = glob.glob("/home/ec2-user/ebs/ocr_data/12009/*.ocr")

In [124]:
len(file_list)

181

In [125]:
ocr_dict = defaultdict(list)
for filename in file_list:
    with open(filename, 'r') as fp:
        page_data = json.load(fp)
        blocks = page_data.get('Blocks', [])
        for block in blocks:
            if 'Text' in block:
                ocr_dict[filename].append({'text':block['Text'], 'geometry':block['Geometry']})

In [126]:
# ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr']

In [127]:
os.getcwd()

'/home/ec2-user/spacy-projects/tutorials/rel_component/scripts'

In [128]:
os.chdir('/home/ec2-user/spacy-projects/tutorials/rel_component/scripts')

In [129]:
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example

# make the factory work
from rel_pipe import make_relation_extractor, score_relations

# make the config work
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [130]:
matcher_lemma = PhraseMatcher(nlp.vocab, attr='LEMMA')
matcher_tags = ["BIRTH",
"MARRIAGE",
"DEATH",
"SPOUSE"
"HUSBAND",
"WIFE",
"CHILD",
"SON",
"DAUGHTER",
"SIBLING",
"BROTHER",
"SISTER",
"PARENT",
"FATHER",
"MOTHER",
"GRANDPARENT",
"GRANDFATHER",
"GRANDMOTHER",
"GRANDCHILD",
"GRANDSON",
"GRANDDAUGHTER",
"COUSIN",
"AUNT",
"UNCLE",
"NIECE",
"NEPHEW"]

pattern_list = [nlp(text) for text in matcher_tags]
matcher_lemma.add("patterns", pattern_list)

In [131]:
for file_name, file_blocks in ocr_dict.items():
    for block in file_blocks:
        text = block['text']
        geometry = block['geometry']
        doc = nlp(text)
        matches = matcher_lemma(doc)
        possible_entity_label_list = []
        for match_id, start, end in matches:
            span = doc[start:end]
            possible_entity_label_list.append(span.text)
        block['possible_entity_labels'] = possible_entity_label_list

In [133]:
# ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr'][10]

In [134]:
# for file_name, file_blocks in ocr_dict.items():
#     for block in file_blocks:
#         if block['possible_entity_labels']:
#             print(block['possible_entity_labels'])

In [135]:
import math

In [136]:
nlp_ner = spacy.load('/home/ec2-user/spacy-projects/tutorials/ner_drugs/training/model-best')

In [138]:
for file_name, file_blocks in ocr_dict.items():
    for block in file_blocks:
        text = block['text']
        geometry = block['geometry']
        possible_entity_labels = block['possible_entity_labels']
        ner_doc = nlp_ner(text)
        ner_list = []
        for ent in ner_doc.ents:
            ner_list.append({'text': ent.text, 'start_char':ent.start_char, 'end_char': ent.end_char, 'start_token': ent.start, 'end_token': ent.end, 'ent_label': ent.label_})
        block['ner_data'] = ner_list
        block['ner_data_raw'] = ner_doc.ents

In [139]:
ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00009-1.ocr']

[{'text': 'Oakley Original Parish Registers',
  'geometry': {'BoundingBox': {'Width': 0.489214688539505,
    'Height': 0.028651047497987747,
    'Left': 0.21233431994915009,
    'Top': 0.13981056213378906},
   'Polygon': [{'X': 0.21303395926952362, 'Y': 0.13981056213378906},
    {'X': 0.7015489935874939, 'Y': 0.1456761658191681},
    {'X': 0.7008493542671204, 'Y': 0.1684616208076477},
    {'X': 0.21233431994915009, 'Y': 0.16259601712226868}]},
  'possible_entity_labels': [],
  'ner_data': [{'text': 'Oakley Original Parish Registers',
    'start_char': 0,
    'end_char': 32,
    'start_token': 0,
    'end_token': 4,
    'ent_label': 'People'}],
  'ner_data_raw': (Oakley Original Parish Registers,)},
 {'text': 'Period searched: 1560-1690 (i.e., from beginning of',
  'geometry': {'BoundingBox': {'Width': 0.6861729025840759,
    'Height': 0.027071604505181313,
    'Left': 0.11197660118341446,
    'Top': 0.17915970087051392},
   'Polygon': [{'X': 0.11255507916212082, 'Y': 0.1791597008705139

In [150]:
ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr']

[{'text': 'The Ancestry of',
  'geometry': {'BoundingBox': {'Width': 0.3044992983341217,
    'Height': 0.03238781914114952,
    'Left': 0.3125407099723816,
    'Top': 0.139822855591774},
   'Polygon': [{'X': 0.31338945031166077, 'Y': 0.139822855591774},
    {'X': 0.6170400381088257, 'Y': 0.1435970515012741},
    {'X': 0.6161912679672241, 'Y': 0.1722106635570526},
    {'X': 0.3125407099723816, 'Y': 0.1684364676475525}]},
  'possible_entity_labels': [],
  'ner_data': [],
  'ner_data_raw': ()},
 {'text': 'THOMAS LOVELL',
  'geometry': {'BoundingBox': {'Width': 0.4671472907066345,
    'Height': 0.03348996117711067,
    'Left': 0.22728750109672546,
    'Top': 0.18067045509815216},
   'Polygon': [{'X': 0.2281089574098587, 'Y': 0.18067045509815216},
    {'X': 0.6944348216056824, 'Y': 0.18646661937236786},
    {'X': 0.6936133503913879, 'Y': 0.21416041254997253},
    {'X': 0.22728750109672546, 'Y': 0.20836424827575684}]},
  'possible_entity_labels': [],
  'ner_data': [{'text': 'THOMAS LOVELL',


In [140]:
trained_pipeline = Path("/home/ec2-user/spacy-projects/tutorials/rel_component/training_1000_epochs/model-best")
nlp_rel = spacy.load(trained_pipeline)

In [162]:
def softmax(confidence_dictionary=None):
    expo_sum_val = 0
    for val in confidence_dictionary.values():
        expo_sum_val += (math.exp(val))
    conf_softmax_dict = dict()
    for key, val in confidence_dictionary.items():
        conf_softmax_dict[key] = (math.exp(val) / expo_sum_val)
    sorted_conf_softmax_dict = {k:v for k, v in sorted(conf_softmax_dict.items(), key = lambda item: item[1], reverse=True)}
    return sorted_conf_softmax_dict

In [163]:
# entity_span_sorted_list = [(low_span, high_span),()]
def extract_entity_overlaps(entity_span_sorted_list=None, doc_tok_index=None, doc=None, low=None, high=None):
    if low <= high:
        mid = (low + high) // 2
        if doc_tok_index >= entity_span_sorted_list[mid][0] and doc_tok_index <= entity_span_sorted_list[mid][1]:
            doc_tok_index_text = doc[entity_span_sorted_list[mid][0]: (entity_span_sorted_list[mid][1] + 1)].text
            return doc_tok_index_text
        elif doc_tok_index < entity_span_sorted_list[mid][0]:
            high = mid - 1
            doc_tok_index_text = extract_entity_overlaps(entity_span_sorted_list=entity_span_sorted_list, doc_tok_index=doc_tok_index, doc=doc, low=low, high=high)
            return doc_tok_index_text
        else:
            low = mid + 1
            doc_tok_index_text = extract_entity_overlaps(entity_span_sorted_list=entity_span_sorted_list, doc_tok_index=doc_tok_index, doc=doc, low=low, high=high)
            return doc_tok_index_text   
    return None

In [ ]:
broken = False
doc_mismatch_exceptions = []
for file_name, file_blocks in ocr_dict.items():
    for block in file_blocks:
        text = block['text']
        geometry = block['geometry']
        possible_entity_labels = block['possible_entity_labels']
        ner_list = block['ner_data']
        ner_raw_data = block['ner_data_raw']
        if ner_raw_data:
            word_tokens = text.split(' ')
            spaces = [' '] * len(word_tokens)
            pred = Doc(nlp_rel.vocab, word_tokens, spaces)
            ner_doc = nlp_ner(text)
            try:
                pred.ents = ner_doc.ents
            except Exception as e:
                doc_mismatch_exceptions.append(str(e))
            entity_span_sorted_list = []
            entity_span_texts = []
            for ent in ner_raw_data:
                entity_span_sorted_list.append((ent.start, ent.end - 1))
                entity_span_texts.append(ent.text)
            entity_span_sorted_list_sorted = sorted(entity_span_sorted_list, key = lambda tup: (tup[0], tup[1]))
            for name, proc in nlp_rel.pipeline:
                pred = proc(pred)
            block['rels'] = dict()
            # came_in = False
            for value, rel_dict in pred._.rel.items():
                token_0_span_text = extract_entity_overlaps(entity_span_sorted_list=entity_span_sorted_list_sorted, doc_tok_index=value[0], doc=ner_doc, low=0, high=len(ner_doc.ents) - 1)
                token_1_span_text = extract_entity_overlaps(entity_span_sorted_list=entity_span_sorted_list_sorted, doc_tok_index=value[1], doc=ner_doc, low=0, high=len(ner_doc.ents) - 1)
                sorted_conf_softmax_dict = softmax(confidence_dictionary=rel_dict)
                for index, (key, val) in enumerate(sorted_conf_softmax_dict.items()):
                    if index < 3:
                        block['rels'][index] = ((token_0_span_text, token_1_span_text), key, val)
                # came_in = True
            # if came_in:
            #     print('Items:', pred._.rel.items())
            #     print('Rels:', block['rels'])
            #     print('span texts:', entity_span_texts)
            #     print('Text:', text)
            #     broken = True
            #     break
    # if broken:
    #     break

ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could no

In [166]:
text_list = []
doc_mismatch_exceptions = []
page_level_docs = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(tuple))))
for file_name, file_blocks in ocr_dict.items():
    for index, block in enumerate(file_blocks):
        text = block['text']
        if len(text.split(' ')) > 1:
            text_list.append(text)
        else:
            if (index + 1) <= (len(file_blocks) - 1):
                if len(file_blocks[index + 1]['text'].split(' ')) > 1:
                    text_list.append(text)
    text_combined = ' '.join(text_list)
    word_tokens = text_combined.split(' ')
    spaces = [' '] * len(word_tokens)
    pred = Doc(nlp_rel.vocab, word_tokens, spaces)
    ner_doc = nlp_ner(text_combined)
    try:
        pred.ents = ner_doc.ents
    except Exception as e:
        doc_mismatch_exceptions.append(str(e))
    entity_span_sorted_list = []
    entity_span_texts = []
    for ent in ner_doc.ents:
        entity_span_sorted_list.append((ent.start, ent.end - 1))
        entity_span_texts.append(ent.text)
    entity_span_sorted_list_sorted = sorted(entity_span_sorted_list, key = lambda tup: (tup[0], tup[1]))
    for name, proc in nlp_rel.pipeline:
        pred = proc(pred)
    # came_in = False
    global_index = 0
    for value, rel_dict in pred._.rel.items():
        token_0_span_text = extract_entity_overlaps(entity_span_sorted_list=entity_span_sorted_list_sorted, doc_tok_index=value[0], doc=ner_doc, low=0, high=len(ner_doc.ents) - 1)
        token_1_span_text = extract_entity_overlaps(entity_span_sorted_list=entity_span_sorted_list_sorted, doc_tok_index=value[1], doc=ner_doc, low=0, high=len(ner_doc.ents) - 1)
        sorted_conf_softmax_dict = softmax(confidence_dictionary=rel_dict)
        for index, (key, val) in enumerate(sorted_conf_softmax_dict.items()):
            if index < 3:
                page_level_docs[file_name]['rels'][global_index][index] = ((token_0_span_text, token_1_span_text), key, val)
        global_index += 1
    page_level_docs[file_name]['all_rels'] = pred._.rel.items()
    #     came_in = True
    # if came_in:
    #     print('Items:', pred._.rel.items())
    #     print('-----------------')
    #     print('Rels:', page_level_docs[file_name]['rels'])
    #     print('-----------------')
    #     print('span texts:', entity_span_texts)
    #     print('-----------------')
    #     print('Text:', text_combined)
    #     print('-----------------')
    #     print('OCR_DICT KEY:')
    #     for block in file_blocks:
    #         print(block)
    #         print("*" * 50)
    #     print('-----------------')
    #     print('File Name:', file_name)
    #     print('-----------------')
    #     break

ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could not determine any instances in doc - returning doc as is.
ℹ Could no

In [168]:
ocr_dict['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr']

[{'text': 'The Ancestry of',
  'geometry': {'BoundingBox': {'Width': 0.3044992983341217,
    'Height': 0.03238781914114952,
    'Left': 0.3125407099723816,
    'Top': 0.139822855591774},
   'Polygon': [{'X': 0.31338945031166077, 'Y': 0.139822855591774},
    {'X': 0.6170400381088257, 'Y': 0.1435970515012741},
    {'X': 0.6161912679672241, 'Y': 0.1722106635570526},
    {'X': 0.3125407099723816, 'Y': 0.1684364676475525}]},
  'possible_entity_labels': [],
  'ner_data': [],
  'ner_data_raw': ()},
 {'text': 'THOMAS LOVELL',
  'geometry': {'BoundingBox': {'Width': 0.4671472907066345,
    'Height': 0.03348996117711067,
    'Left': 0.22728750109672546,
    'Top': 0.18067045509815216},
   'Polygon': [{'X': 0.2281089574098587, 'Y': 0.18067045509815216},
    {'X': 0.6944348216056824, 'Y': 0.18646661937236786},
    {'X': 0.6936133503913879, 'Y': 0.21416041254997253},
    {'X': 0.22728750109672546, 'Y': 0.20836424827575684}]},
  'possible_entity_labels': [],
  'ner_data': [{'text': 'THOMAS LOVELL',


In [169]:
page_level_docs['/home/ec2-user/ebs/ocr_data/12009/12009_dvm_dez_GenMono001490-00001-1.ocr']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'rels': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                         {0: defaultdict(tuple,
                                      {0: (('THOMAS LOVELL', 'his'),
                                        'self',
                                        0.14383724134113635),
                                       1: (('THOMAS LOVELL', 'his'),
                                        'daughter',
                                        0.14269379310981065),
                                       2: (('THOMAS LOVELL', 'his'),
                                        'father',
                                        0.14269379310981065)}),
                          1: defaultdict(tuple,
                                      {0: (('THOMAS LOVELL',
                                         'MARY ELLEN RICKER'),
                                        'wife',
                        

In [174]:
def make_sentence_level_processing_serializable(ocr_dict = None):
    serializable_dictionary = dict()
    for file_name, file_blocks in ocr_dict.items():
        local_blocks = []
        for block in file_blocks:
            hyper_local_dict = dict()
            for key, val in block.items():
                if key != 'ner_data_raw':
                    hyper_local_dict[key] = val
            local_blocks.append(hyper_local_dict)
        serializable_dictionary[file_name] = local_blocks
    return serializable_dictionary

In [184]:
sentence_level_serializable_dictionary = make_sentence_level_processing_serializable(ocr_dict = ocr_dict)

In [187]:
def make_page_level_processing_serializable(page_level_docs = None):
    serializable_dictionary = defaultdict(dict)
    for file_name, page_dict in page_level_docs.items():
        serializable_dictionary[file_name]['relations'] = page_dict['rels']
    return serializable_dictionary

In [188]:
page_level_serializable_dictionary = make_page_level_processing_serializable(page_level_docs = page_level_docs)

In [189]:
with open('book_12009_sentence_level_data_processing.json', 'w') as fp:
    json.dump(sentence_level_serializable_dictionary, fp, indent=4)

In [190]:
with open('book_12009_page_level_data_processing.json', 'w') as fp:
    json.dump(page_level_serializable_dictionary, fp, indent=4)

In [191]:
import os

In [192]:
os.getcwd()

'/home/ec2-user/spacy-projects/tutorials/rel_component/scripts'

In [193]:
# ------ Rough ------

In [57]:
a = "Karthik Ragunath is a great guy"
doc = nlp(a)

In [61]:
doc[1:1].text

''

In [70]:
doc_ents = nlp(a).ents
for ent in doc_ents:
    print(ent.start, ent.end, ent.text)

0 2 Karthik Ragunath


In [71]:
nlp(a)[0].text

'Karthik'

In [72]:
nlp(a)[1].text

'Ragunath'

In [73]:
len(doc)

6

In [75]:
nlp(a)[5].text

'guy'

In [114]:
for index, (key, val) in enumerate({1:2, 3:4, 5:6, 7:8, 9:10}.items()):
    print("Index:", index, "Key:", key, "Value:", val)

Index: 0 Key: 1 Value: 2
Index: 1 Key: 3 Value: 4
Index: 2 Key: 5 Value: 6
Index: 3 Key: 7 Value: 8
Index: 4 Key: 9 Value: 10


In [164]:
softmax({1: 0.5, 2: 0.25, 3: 0.25})

{1: 0.390991315159432, 2: 0.304504342420284, 3: 0.304504342420284}